In [1]:
import numpy as np
from gensim.models import Word2Vec
from keras.callbacks import Callback, EarlyStopping
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D, CuDNNGRU, CuDNNLSTM, GRU, LSTM, Reshape, TimeDistributed
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
#from keras_self_attention import SeqSelfAttention
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, hamming_loss, f1_score
from sklearn.preprocessing import MultiLabelBinarizer
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from tqdm import tnrange, tqdm_notebook
from utils import process_data, multilabel_confusion_matrix, get_embedding_matrix, get_cat_labels, data_generator, get_all

# Customized Evaluation
class CustomEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = list(validation_data)

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = []
            for x in self.X_val:
                y = np.squeeze(self.model.predict_on_batch(x))
                y_pred.append(y)
            y_pred = np.concatenate(y_pred)
            y_pred_ham = y_pred > 0.5
            y_val = np.concatenate(self.y_val)
            roc = roc_auc_score(y_val, y_pred, average='micro')
            loss = log_loss(y_val, y_pred)
            ham = hamming_loss(y_val, y_pred_ham)
            sub = accuracy_score(y_val, y_pred_ham)
            f1 = f1_score(y_val, y_pred_ham, average='micro')
            print("Adiitional val metrics: - ROC-AUC: %.6f - Log-Loss: %.6f - Hamming-Loss: %.6f - Subset-Accuracy: %.6f - F1-Score: %.6f" % (roc, loss, ham, sub, f1))

def no_pad_time_tuning(param, notes_train, labels_train, up_notes_train, up_labels_train, gold_labels_train, notes_test, labels_test, gold_labels_test, verbose=1):
    
    up = param['up']
    window_size = param['window_size']
    embed_size = param['embed_size'] * 10
    latent_dim = param['latent_dim'] * 64
    dropout_rate = param['dropout_rate']
    epochs = 30 #param['epochs']
    max_features = 60000 #param['max_features']
    category = False #param['category']
    embedding = True #param['embedding']
    model_type = 'CuDNNLSTM' #param['model_type']
    
    # upsampling
    if up > 0:
        if verbose != 0: print('upsampling for %d times...' % (up))
        notes_train = [note + up * up_note for note, up_note in zip(notes_train, up_notes_train)]
        labels_train = [label + up * up_label for label, up_label in zip(labels_train, up_labels_train)]
        if verbose != 0: print('upsampling done\n')
    notes = notes_train + notes_test
    labels = labels_train + labels_test
    gold_labels = gold_labels_train + gold_labels_test
    
    # prepare features
    if verbose != 0: print('preparing features ...')
    X_txt = [' '.join(i) for i in notes]
    X_train_txt = [' '.join(i) for i in notes_train]
    X_test_txt = [' '.join(i) for i in notes_test]
    tokenizer = Tokenizer(num_words=max_features, filters='')
    tokenizer.fit_on_texts(X_txt)
    X_seq = tokenizer.texts_to_sequences(X_txt) 
    X_train_seq = tokenizer.texts_to_sequences(X_train_txt) 
    X_test_seq = tokenizer.texts_to_sequences(X_test_txt) 
    word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    if verbose != 0: print('preparing features done\n')

    # prepare embedding matrix
    if embedding:
        if verbose != 0: print('preparing embedding matrix ...')
        w2v = Word2Vec(notes, size=embed_size, window=window_size, min_count=1, workers=4)
        embedding_index = dict(zip(w2v.wv.index2word, w2v.wv.vectors))
        embedding_matrix = get_embedding_matrix(embedding_index=embedding_index, word_index=word_index, max_features=max_features, embed_size=embed_size)
        if verbose != 0: print('preparing embedding matrix done\n')
        
    # prepare targets
    if verbose != 0: print('preparing targets ...')
    if category:
        # prepare cagtegory label targets
        labels = [[set([get_cat_labels(i) for i in list(j)]) for j in k] for k in labels]
        labels_train = [[set([get_cat_labels(i) for i in list(j)]) for j in k] for k in labels_train]
        labels_test = [[set([get_cat_labels(i) for i in list(j)]) for j in k] for k in labels_test]
    all_labels = [label for notes_label in labels for label in notes_label]
    mlb = MultiLabelBinarizer()
    mlb.fit(all_labels)
    num_labels = len(mlb.classes_)
    Y_train = []
    Y_test = []
    for i in labels_train:
        l = mlb.transform(i)
        Y_train.append(l)
    for i in labels_test:
        l = mlb.transform(i)
        Y_test.append(l)
    if verbose != 0: print('preparing targets done\n')

    # model function with pretrained embedding matrix and Timedistributed
    def get_model(nb_words, num_labels, model_type, embedding):
        inp = Input(shape=(None, ))
        if embedding:
            x = Embedding(nb_words, embed_size, weights=[embedding_matrix])(inp)
        else:    
            x = Embedding(nb_words, embed_size)(inp)
        x = SpatialDropout1D(dropout_rate)(x)
        if model_type=='CuDNNGRU':
            x = Bidirectional(CuDNNGRU(latent_dim, return_sequences=True))(x)
        elif model_type=='GRU':
            x = Bidirectional(GRU(latent_dim, return_sequences=True))(x)
        elif model_type=='CuDNNLSTM':
            x = Bidirectional(CuDNNLSTM(latent_dim, return_sequences=True))(x)
        elif model_type=='LSTM':
            x = Bidirectional(LSTM(latent_dim, return_sequences=True))(x)
        else:
            raise ValueError('Please specify model_type as one of the following:n\CuDNNGRU, CuDNNLSTM, GRU, LSTM')
        #x = SeqSelfAttention(attention_width=15, attention_activation='sigmoid')(x)
        outp = TimeDistributed(Dense((num_labels), activation="sigmoid"))(x)

        model = Model(inputs=inp, outputs=outp)
        model.compile(loss='binary_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])

        return model

    # model summary
    model = get_model(nb_words=nb_words, num_labels=num_labels, model_type=model_type, embedding=embedding)
    if verbose != 0: print('\nmodel summary:')
    if verbose != 0: print(model.summary())

    # model training
    if verbose != 0: print('\ntraining model ...')
    custevl = CustomEvaluation(validation_data=(X_test_seq, Y_test), interval=1)
    earlystop = EarlyStopping(monitor='val_loss', min_delta=3e-4, patience=3, verbose=0, mode='auto')
    train_gen = data_generator(X_train_seq, Y_train)
    test_gen = data_generator(X_test_seq, Y_test)
    v = 1 if verbose != 0 else 0  
    hist = model.fit_generator(train_gen,
                                steps_per_epoch=len(Y_train),
                                epochs=epochs,
                                validation_data=test_gen,
                                validation_steps=len(Y_test),
                                callbacks=[custevl, earlystop],
                                verbose=v)
    if verbose != 0: print('training model done')

    # prediction of test data
    if verbose != 0: print('predicting test data ...')
    Y_pred = []
    for x in X_test_seq:
        x = np.array(x).reshape((1,-1))
        y_pred = np.squeeze(model.predict_on_batch(x))
        Y_pred.append(y_pred)
    Y_pred_concat = np.concatenate(Y_pred)
    Y_val = np.concatenate(Y_test)
    if verbose != 0: print('predicting test data done\n')

    # confusion matrix 
    if verbose == 2: 
        cm = multilabel_confusion_matrix(Y_val, np.where(Y_pred_concat > 0.5, 1, 0))
        for i, j in zip(cm, mlb.classes_):
            print(j+':\n', i,'\n')

    # prepare gold label targets
    if verbose != 0: print('predicting gold label targets ...')
    gold_labels_pred = [{i for s in mlb.inverse_transform(y_pred>0.5) for i in s if i != 'O'} for y_pred in Y_pred]
    gmlb = MultiLabelBinarizer()
    gmlb.fit(gold_labels)
    num_gold_labels = len(gmlb.classes_)
    Y_gold_test = gmlb.transform(gold_labels_test)
    Y_gold_pred = gmlb.transform(gold_labels_pred)
    if verbose != 0: print('predicting gold label targets done\n')

    # confusion matrix for gold label
    if verbose == 2: 
        gcm = multilabel_confusion_matrix(np.concatenate(Y_gold_test), np.concatenate(Y_gold_pred))
        for i, j in zip(gcm, gmlb.classes_):
            print(j+':\n', i,'\n')

    # f1 scores for gold label
    f1 = f1_score(Y_gold_test, Y_gold_pred, average='micro')
    
    print('Parameters: up - %d, window_size - %d, embed_size - %d, latent_dim - %d, dropout_rate - %.3f' % (up, window_size, embed_size, latent_dim, dropout_rate))
    print('F1 Scores for global labels:\nALL (average="micro"):', f1
    
    if verbose == 2: 
        f1_all = f1_score(Y_gold_test, Y_gold_pred, average=None)
        for i, j in zip(f1_all, gmlb.classes_):
            print(j+': '+str(i))
    
    print('\n\n')
          
    return f1

def my_bayes_opt(space):
    param = {
        'up': space[0],               # Times of upsampling for training data
        'window_size': space[1],                # Window size for word2vec
        'embed_size': space[2],                # Length of the vector that we willl get from the embedding layer
        'latent_dim': space[3],               # Hidden layers dimension 
        'dropout_rate': space[4]}#,             # Rate of the dropout layers
        #'epochs': space[0],                    # Number of epochs
        #'max_features': space[0],           # Max num of vocabulary
        #'category': space[0],               # Is categoty labels
        #'embedding': space[0],               # Using pre-made embedidng matrix as weight
        #'model_type': space[0]
        #}
    f1 = no_pad_time_tuning(param, notes_train, labels_train, up_notes_train, up_labels_train, gold_labels_train, notes_test, labels_test, gold_labels_test)
    return (-f1)     

Using TensorFlow backend.


In [2]:
# loading data 
notes_train_1, labels_train_1, up_notes_train_1, up_labels_train_1, gold_labels_train_1 = get_all('/host_home/data/i2b2/2014/training/training-RiskFactors-Complete-Set1') 
notes_train_2, labels_train_2, up_notes_train_2, up_labels_train_2, gold_labels_train_2 = get_all('/host_home/data/i2b2/2014/training/training-RiskFactors-Complete-Set2') 

notes_train = notes_train_1 + notes_train_2
labels_train = labels_train_1 + labels_train_2
up_notes_train = up_notes_train_1 + up_notes_train_2
up_labels_train = up_labels_train_1 + up_labels_train_2
gold_labels_train = gold_labels_train_1 + gold_labels_train_2

notes_test, labels_test, _1, _2, gold_labels_test = get_all('/host_home/data/i2b2/2014/testing/testing-RiskFactors-Complete') 

In [3]:
space = [Integer(5, 10, name='up'),
        Integer(3, 5, name='window_size'),
        Integer(2, 4, name='embed_size'),
        Integer(1, 3, name='latent_dim'),
        Real(0, 0.3, name='dropout_rate')]
        #Integer(30, 30, name='epochs'),
        #Integer(1, 60000, name='max_features'),
        #Categorical([False], name='category'),
        #Categorical([True], name='embedding'),
        #Categorical(['CuDNNLSTM'], name='model_type')]

res = gp_minimize(my_bayes_opt, space, n_calls=10)

upsampling for 3 times...
upsampling done

preparing features ...
preparing features done

preparing embedding matrix ...
preparing embedding matrix done

preparing targets ...
preparing targets done

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

model summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 10)          449840    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, None, 10)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 128)         38912     
__

790/790 [==============================] - 84s 106ms/step - loss: 0.0162 - acc: 0.9964 - val_loss: 0.0065 - val_acc: 0.9987
Adiitional val metrics: - ROC-AUC: 0.972094 - Log-Loss: 3.800931 - Hamming-Loss: 0.015130 - Subset-Accuracy: 0.759985 - F1-Score: 0.549554
Epoch 4/30
790/790 [==============================] - 85s 108ms/step - loss: 0.0147 - acc: 0.9966 - val_loss: 0.0059 - val_acc: 0.9988
Adiitional val metrics: - ROC-AUC: 0.975618 - Log-Loss: 3.608239 - Hamming-Loss: 0.013253 - Subset-Accuracy: 0.784751 - F1-Score: 0.584723
Epoch 5/30
790/790 [==============================] - 87s 110ms/step - loss: 0.0136 - acc: 0.9968 - val_loss: 0.0055 - val_acc: 0.9988
Adiitional val metrics: - ROC-AUC: 0.977753 - Log-Loss: 3.402556 - Hamming-Loss: 0.011723 - Subset-Accuracy: 0.794126 - F1-Score: 0.613310
Epoch 6/30
790/790 [==============================] - 89s 113ms/step - loss: 0.0126 - acc: 0.9970 - val_loss: 0.0053 - val_acc: 0.9989
Adiitional val metrics: - ROC-AUC: 0.979340 - Log-Loss

790/790 [==============================] - 56s 70ms/step - loss: 0.0048 - acc: 0.9986 - val_loss: 0.0040 - val_acc: 0.9990
Adiitional val metrics: - ROC-AUC: 0.987511 - Log-Loss: 1.936830 - Hamming-Loss: 0.002337 - Subset-Accuracy: 0.901577 - F1-Score: 0.892219
Epoch 14/30
790/790 [==============================] - 56s 71ms/step - loss: 0.0046 - acc: 0.9987 - val_loss: 0.0040 - val_acc: 0.9990
Adiitional val metrics: - ROC-AUC: 0.987918 - Log-Loss: 1.832656 - Hamming-Loss: 0.002084 - Subset-Accuracy: 0.907610 - F1-Score: 0.903416
training model done
predicting test data ...
predicting test data done

predicting gold label targets ...
predicting gold label targets done

F1 Scores:
ALL (average="micro"): 0.8441121870453063
upsampling for 7 times...
upsampling done

preparing features ...
preparing features done

preparing embedding matrix ...
preparing embedding matrix done

preparing targets ...
preparing targets done


model summary:
____________________________________________________

Adiitional val metrics: - ROC-AUC: 0.964474 - Log-Loss: 3.851392 - Hamming-Loss: 0.004469 - Subset-Accuracy: 0.889381 - F1-Score: 0.812655
Epoch 2/30
790/790 [==============================] - 73s 93ms/step - loss: 0.0157 - acc: 0.9968 - val_loss: 0.0073 - val_acc: 0.9987
Adiitional val metrics: - ROC-AUC: 0.973580 - Log-Loss: 3.586821 - Hamming-Loss: 0.004794 - Subset-Accuracy: 0.874995 - F1-Score: 0.801297
Epoch 3/30
790/790 [==============================] - 73s 92ms/step - loss: 0.0129 - acc: 0.9971 - val_loss: 0.0061 - val_acc: 0.9988
Adiitional val metrics: - ROC-AUC: 0.980274 - Log-Loss: 3.235216 - Hamming-Loss: 0.003930 - Subset-Accuracy: 0.899599 - F1-Score: 0.833437
Epoch 4/30
790/790 [==============================] - 72s 91ms/step - loss: 0.0110 - acc: 0.9974 - val_loss: 0.0053 - val_acc: 0.9989
Adiitional val metrics: - ROC-AUC: 0.983295 - Log-Loss: 2.950580 - Hamming-Loss: 0.003048 - Subset-Accuracy: 0.911326 - F1-Score: 0.867387
Epoch 5/30
790/790 [======================

Epoch 1/30
790/790 [==============================] - 91s 115ms/step - loss: 0.0316 - acc: 0.9923 - val_loss: 0.0088 - val_acc: 0.9986
Adiitional val metrics: - ROC-AUC: 0.964542 - Log-Loss: 4.042624 - Hamming-Loss: 0.020103 - Subset-Accuracy: 0.824664 - F1-Score: 0.481644
Epoch 2/30
790/790 [==============================] - 91s 116ms/step - loss: 0.0172 - acc: 0.9963 - val_loss: 0.0070 - val_acc: 0.9987
Adiitional val metrics: - ROC-AUC: 0.964541 - Log-Loss: 3.810406 - Hamming-Loss: 0.023511 - Subset-Accuracy: 0.771179 - F1-Score: 0.428871
Epoch 3/30
790/790 [==============================] - 93s 118ms/step - loss: 0.0149 - acc: 0.9965 - val_loss: 0.0059 - val_acc: 0.9988
Adiitional val metrics: - ROC-AUC: 0.970024 - Log-Loss: 3.557257 - Hamming-Loss: 0.022578 - Subset-Accuracy: 0.799911 - F1-Score: 0.448079
Epoch 4/30
790/790 [==============================] - 96s 121ms/step - loss: 0.0133 - acc: 0.9968 - val_loss: 0.0054 - val_acc: 0.9989
Adiitional val metrics: - ROC-AUC: 0.974592

790/790 [==============================] - 84s 107ms/step - loss: 0.0120 - acc: 0.9970 - val_loss: 0.0050 - val_acc: 0.9989
Adiitional val metrics: - ROC-AUC: 0.986568 - Log-Loss: 2.382436 - Hamming-Loss: 0.005763 - Subset-Accuracy: 0.774508 - F1-Score: 0.750243
Epoch 14/30
790/790 [==============================] - 86s 108ms/step - loss: 0.0115 - acc: 0.9971 - val_loss: 0.0050 - val_acc: 0.9989
Adiitional val metrics: - ROC-AUC: 0.987360 - Log-Loss: 2.259347 - Hamming-Loss: 0.004900 - Subset-Accuracy: 0.789833 - F1-Score: 0.779235
Epoch 15/30
790/790 [==============================] - 85s 107ms/step - loss: 0.0111 - acc: 0.9971 - val_loss: 0.0049 - val_acc: 0.9989
Adiitional val metrics: - ROC-AUC: 0.987953 - Log-Loss: 2.213219 - Hamming-Loss: 0.005061 - Subset-Accuracy: 0.773180 - F1-Score: 0.770882
training model done
predicting test data ...
predicting test data done

predicting gold label targets ...
predicting gold label targets done

F1 Scores:
ALL (average="micro"): 0.690347039

In [4]:
res.x

[7, 4, 3, 2, 0.15253569878187465]

In [5]:
res.fun

-0.8647624599242203

In [37]:
# parameter:
param = {
        'up': 2,                         # Times of upsampling for training data
        'window_size': 5,                # Window size for word2vec
        'embed_size': 10,                # Length of the vector that we willl get from the embedding layer
        'latent_dim': 128,               # Hidden layers dimension 
        'dropout_rate': 0.5,             # Rate of the dropout layers
        'epochs': 30,                    # Number of epochs
        'max_features': 60000,           # Max num of vocabulary
        'category': False,               # Is categoty labels
        'embedding': True,               # Using pre-made embedidng matrix as weight
        'model_type': 'CuDNNLSTM'
        }

In [43]:
f1 = no_pad_time_tuning(param, notes_train, labels_train, up_notes_train, up_labels_train, gold_labels_train, notes_test, labels_test, gold_labels_test)

upsampling for 2 times...
upsampling done

preparing features ...
preparing features done

preparing embedding matrix ...
preparing embedding matrix done

preparing targets ...
preparing targets done


model summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_56 (InputLayer)        (None, None)              0         
_________________________________________________________________
embedding_56 (Embedding)     (None, None, 10)          449840    
_________________________________________________________________
spatial_dropout1d_56 (Spatia (None, None, 10)          0         
_________________________________________________________________
bidirectional_56 (Bidirectio (None, None, 256)         143360    
_________________________________________________________________
seq_self_attention_4 (SeqSel (None, None, 256)         16449     
_________________________________________________________

predicting gold label targets done

I.CAD.after_DCT.event:
 [[  998  9307]
 [  174 38865]] 

I.CAD.after_DCT.mention:
 [[38865   174]
 [ 9307   998]] 

F1 Scores:
ALL (average="micro"): 0.1739130434782609
I.CAD.after_DCT.event: 0.0
I.CAD.after_DCT.mention: 0.47787610619469034
I.CAD.after_DCT.symptom: 0.0
I.CAD.before_DCT.event: 0.0
I.CAD.before_DCT.mention: 0.4978165938864629
I.CAD.before_DCT.symptom: 0.0
I.CAD.before_DCT.test: 0.5319148936170213
I.CAD.during_DCT.event: 0.0
I.CAD.during_DCT.mention: 0.47413793103448276
I.CAD.during_DCT.symptom: 0.0
I.CAD.during_DCT.test: 0.0
I.DIABETES.after_DCT.mention: 0.7346221441124782
I.DIABETES.before_DCT.A1C: 0.0
I.DIABETES.before_DCT.glucose: 0.0
I.DIABETES.before_DCT.mention: 0.7288732394366196
I.DIABETES.during_DCT.A1C: 0.0
I.DIABETES.during_DCT.glucose: 0.0
I.DIABETES.during_DCT.mention: 0.7412587412587414
I.FAMILY_HIST.present: 0.0
I.HYPERLIPIDEMIA.after_DCT.mention: 0.0
I.HYPERLIPIDEMIA.before_DCT.high_LDL: 0.0
I.HYPERLIPIDEMIA.before_DCT.

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [44]:
f1

0.1739130434782609

In [ ]:
f1_all

In [26]:
# model training batch by batch
epochs = 10
patience = 3
threshhold = 0.001
loss_n = []
for epoch in tnrange(epochs, desc='total progress'):
    for x, y in tqdm_notebook(zip(X_train_seq, Y_train), desc='epoch '+str(epoch+1)+' training', total=len(Y_train)):
        ly = len(y)
        x = np.array(x).reshape((1,-1))
        y = np.array(y).reshape((1,ly,-1))
        model.train_on_batch(x, y)
    Y_pred = []
    for x in tqdm_notebook(X_test_seq, desc='epoch '+str(epoch+1)+' validating'):
        x = np.array(x).reshape((1,-1))
        y_pred = np.squeeze(model.predict_on_batch(x))
        Y_pred.append(y_pred)
    Y_pred = np.concatenate(Y_pred)
    Y_pred_ham = Y_pred > 0.5
    Y_val = np.concatenate(Y_test)
    loss = log_loss(Y_val, Y_pred)
    ham = hamming_loss(Y_val, Y_pred_ham)
    sub = accuracy_score(Y_val, Y_pred_ham)
    f1 = f1_score(Y_val, Y_pred_ham, average='micro')
    print("Epoch %d val metrics: - Log-Loss: %.6f - Hamming-Loss: %.6f - Subset-Accuracy: %.6f - F1-Score: %.6f" % (epoch, loss, ham, sub, f1))
    loss_n.append(loss)
    if len(loss_n) > patience:
        for i in loss_n[-4:]:
            if loss_n[-1] >= i + threshhold:
                break

Epoch 0 val metrics: - Log-Loss: 0.351766 - Hamming-Loss: 0.001083 - Subset-Accuracy: 0.957040 - F1-Score: 0.949087


Epoch 1 val metrics: - Log-Loss: 0.347610 - Hamming-Loss: 0.001072 - Subset-Accuracy: 0.957779 - F1-Score: 0.949651


Epoch 2 val metrics: - Log-Loss: 0.337398 - Hamming-Loss: 0.001058 - Subset-Accuracy: 0.957161 - F1-Score: 0.950335


Epoch 3 val metrics: - Log-Loss: 0.333103 - Hamming-Loss: 0.001052 - Subset-Accuracy: 0.956803 - F1-Score: 0.950598


Epoch 4 val metrics: - Log-Loss: 0.322978 - Hamming-Loss: 0.001027 - Subset-Accuracy: 0.956368 - F1-Score: 0.951716


Epoch 5 val metrics: - Log-Loss: 0.321765 - Hamming-Loss: 0.001019 - Subset-Accuracy: 0.956749 - F1-Score: 0.952139


Epoch 6 val metrics: - Log-Loss: 0.319472 - Hamming-Loss: 0.001020 - Subset-Accuracy: 0.957300 - F1-Score: 0.952150


Epoch 7 val metrics: - Log-Loss: 0.311171 - Hamming-Loss: 0.001006 - Subset-Accuracy: 0.956390 - F1-Score: 0.952800


Epoch 8 val metrics: - Log-Loss: 0.312243 - Hamming-Loss: 0.001003 - Subset-Accuracy: 0.956964 - F1-Score: 0.952942


Epoch 9 val metrics: - Log-Loss: 0.304512 - Hamming-Loss: 0.000993 - Subset-Accuracy: 0.957218 - F1-Score: 0.953429


In [54]:
from tqdm import tnrange, tqdm_notebook
epochs = 10
for epoch in tnrange(epochs):
    for x, y in tqdm_notebook(zip(X_train_seq, Y_cat_train), desc='training'):
        ly = len(y)
        x = np.array(x).reshape((1,-1))
        y = np.array(y).reshape((1,ly,-1))
        #if x.shape[1] != y.shape[1]:
        #    print(x.shape,y.shape)
        cat_model.train_on_batch(x, y)
    Y_cat_pred = []
    for x in tqdm_notebook(X_test_seq, desc='predicting'):
        x = np.array(x).reshape((1,-1))
        #if x.shape[1] != y.shape[1]:
        #    print(x.shape,y.shape)
        y_pred = np.squeeze(cat_model.predict_on_batch(x))
        Y_cat_pred.append(y_pred)
    Y_cat_pred = np.concatenate(Y_cat_pred)
    Y_cat_pred_ham = Y_cat_pred > 0.5
    Y_cat_val = np.concatenate(Y_cat_test)
    #print(y_val.sum(), y_pred.sum())
    #roc = roc_auc_score(Y_val, Y_pred)
    loss = log_loss(Y_cat_val, Y_cat_pred)
    ham = hamming_loss(Y_cat_val, Y_cat_pred_ham)
    sub = accuracy_score(Y_cat_val, Y_cat_pred_ham)
    f1 = f1_score(Y_val, Y_pred_ham, average='micro')
    print("Epoch %d val metrics: - Log-Loss: %.6f - Hamming-Loss: %.6f - Subset-Accuracy: %.6f - F1_Score: %.6f" % (epoch, loss, ham, sub, f1))

Epoch 0 val metrics: - Log-Loss: 0.239218 - Hamming-Loss: 0.010005 - Subset-Accuracy: 0.954910


Epoch 1 val metrics: - Log-Loss: 0.212364 - Hamming-Loss: 0.009780 - Subset-Accuracy: 0.955373


Epoch 2 val metrics: - Log-Loss: 0.196264 - Hamming-Loss: 0.009550 - Subset-Accuracy: 0.955572


Epoch 3 val metrics: - Log-Loss: 0.179833 - Hamming-Loss: 0.009293 - Subset-Accuracy: 0.955243


Epoch 4 val metrics: - Log-Loss: 0.165139 - Hamming-Loss: 0.008795 - Subset-Accuracy: 0.956336


Epoch 5 val metrics: - Log-Loss: 0.154288 - Hamming-Loss: 0.008518 - Subset-Accuracy: 0.956742


Epoch 6 val metrics: - Log-Loss: 0.148316 - Hamming-Loss: 0.008245 - Subset-Accuracy: 0.957681


Epoch 7 val metrics: - Log-Loss: 0.141742 - Hamming-Loss: 0.008039 - Subset-Accuracy: 0.958841


Epoch 8 val metrics: - Log-Loss: 0.137761 - Hamming-Loss: 0.007891 - Subset-Accuracy: 0.959345


Epoch 9 val metrics: - Log-Loss: 0.135815 - Hamming-Loss: 0.007833 - Subset-Accuracy: 0.959215
